# 数据处理模块

## 目录

* 数据集加载
* 分句，分词以及划分数据集
* 加载训练集
* 构建word2id，char2id
* 构建特征和标签
* 生成torch数据导入类

In [4]:
import  json
import nltk

In [5]:
# 数据集加载
datas = open("./data/wiki_00",encoding="utf-8").read().splitlines()
datas[0:5]

FileNotFoundError: [Errno 2] No such file or directory: './data/wiki_00'

In [ ]:
json_data = json.loads(datas[0],strict=False)
json_data

In [ ]:
text_data = json_data["text"]
text_data

In [ ]:
text_data = text_data.replace("\n\n",". ")
text_data = text_data.replace("\n",". ")
text_data

In [ ]:
text_data = nltk.sent_tokenize(text_data)
text_data

In [ ]:
tokens = nltk.word_tokenize(text_data[-1])
tokens

In [ ]:
# 分句，分词以及划分数据集
num_words = 0
f_train = open("./data/train.txt","w",encoding="utf-8")
f_valid = open("./data/valid.txt","w",encoding="utf-8")
f_test = open("./data/test.txt","w",encoding="utf-8")
for i,data in enumerate(datas):
    if i%1000==0:
        print (i,len(datas))
    data = json.loads(data,strict=False)
    sentences = data["text"]
    sentences = sentences.replace("\n\n",". ")
    sentences = sentences.replace("\n",". ")
    sentences = nltk.sent_tokenize(sentences)
    for sentence in sentences:
        sentence = nltk.word_tokenize(sentence)
        if len(sentence)<10 or len(sentence)>100:
            continue
        num_words+=len(sentence)
        sentence = " ".join(sentence) +"\n"
        if num_words<=1000000:
            f_train.write(sentence)
        elif num_words<=1020000:
            f_valid.write(sentence)
        elif num_words<=1040000:
            f_test.write(sentence)
        else:
            break


In [3]:
# 加载训练集
import numpy as np
datas = open("./data/train.txt",encoding="utf-8").read().strip("\n").splitlines()
datas = [s.split() for s in datas]
np.array(datas[0:5])

/var/folders/rj/kh58t7hj5s99k8krgn4brl1c0000gn/T/ipykernel_372/2677487255.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(datas[0:5])


array([list(['Konica', 'Cup', '(', 'before', 'the', 'Minolta', 'merger', ')', 'may', 'refer', 'to', '.']),
       list(['Archer', 'is', 'a', 'slab', 'serif', 'typeface', 'designed', 'in', '2001', 'by', 'Tobias', 'Frere-Jones', 'and', 'Jonathan', 'Hoefler', 'for', 'use', 'in', '``', 'Martha', 'Stewart', 'Living', "''", 'magazine', '.']),
       list(['It', 'was', 'later', 'released', 'by', 'Hoefler', '&', 'Frere-Jones', 'for', 'commercial', 'licensing..']),
       list(['The', 'typeface', 'is', 'a', 'geometric', 'slab', 'serif', ',', 'one', 'with', 'a', 'geometric', 'design', 'similar', 'to', 'sans-serif', 'fonts', '.']),
       list(['It', 'takes', 'inspiration', 'from', 'mid-twentieth', 'century', 'designs', 'such', 'as', 'Rockwell..'])],
      dtype=object)

In [86]:
# 构建word2id，char2id
from collections import Counter

In [87]:
words = []
chars = []
for data in datas:
    for word in data:
        words.append(word.lower())
        chars.extend(word)
print (len(words))
print (len(chars))

999539
4511117


In [88]:
words = dict(Counter(words).most_common(5000-2))
chars = dict(Counter(chars).most_common(512-3))
print (words)
print (chars)

{'the': 62903, ',': 52369, '.': 30394, 'of': 28911, 'and': 26721, 'in': 25847, 'to': 19704, 'a': 18797, 'was': 11451, '``': 10115, "''": 9838, 'is': 8063, 'for': 7812, 'on': 7609, 'as': 7493, 'by': 6448, '(': 6430, 'with': 6430, ')': 6426, 'he': 6149, 'that': 5711, "'s": 5444, 'at': 4961, 'from': 4713, 'his': 4697, 'it': 3964, 'an': 3666, 'were': 2800, 'which': 2648, 'are': 2593, 'this': 2427, 'be': 2415, 'also': 2278, 'had': 2266, 'their': 2070, 'has': 2031, 'first': 2013, 'her': 1967, 'after': 1941, 'or': 1890, 'not': 1807, 'she': 1799, 'one': 1765, 'but': 1719, 'new': 1719, 'they': 1661, 'its': 1657, 'who': 1642, 'have': 1543, 'been': 1452, ':': 1414, ';': 1413, 'two': 1401, 'when': 1314, 'during': 1284, 'all': 1201, 'other': 1181, 'school': 1169, 'into': 1090, 'would': 1085, 'time': 1039, 'there': 1024, '..': 997, 'may': 986, 'him': 974, 'united': 959, 'more': 959, 'where': 911, 'some': 903, 'only': 899, 'state': 894, 'can': 890, 'city': 876, 'over': 875, 'most': 874, "'": 867, 'st

In [89]:
print (len(words))
print (len(chars))

4998
509


In [92]:
word2id = {"<pad>":0,"<unk>":1}
for word in words:
    word2id[word] = len(word2id)
word2id

{'<pad>': 0,
 '<unk>': 1,
 'the': 2,
 ',': 3,
 '.': 4,
 'of': 5,
 'and': 6,
 'in': 7,
 'to': 8,
 'a': 9,
 'was': 10,
 '``': 11,
 "''": 12,
 'is': 13,
 'for': 14,
 'on': 15,
 'as': 16,
 'by': 17,
 '(': 18,
 'with': 19,
 ')': 20,
 'he': 21,
 'that': 22,
 "'s": 23,
 'at': 24,
 'from': 25,
 'his': 26,
 'it': 27,
 'an': 28,
 'were': 29,
 'which': 30,
 'are': 31,
 'this': 32,
 'be': 33,
 'also': 34,
 'had': 35,
 'their': 36,
 'has': 37,
 'first': 38,
 'her': 39,
 'after': 40,
 'or': 41,
 'not': 42,
 'she': 43,
 'one': 44,
 'but': 45,
 'new': 46,
 'they': 47,
 'its': 48,
 'who': 49,
 'have': 50,
 'been': 51,
 ':': 52,
 ';': 53,
 'two': 54,
 'when': 55,
 'during': 56,
 'all': 57,
 'other': 58,
 'school': 59,
 'into': 60,
 'would': 61,
 'time': 62,
 'there': 63,
 '..': 64,
 'may': 65,
 'him': 66,
 'united': 67,
 'more': 68,
 'where': 69,
 'some': 70,
 'only': 71,
 'state': 72,
 'can': 73,
 'city': 74,
 'over': 75,
 'most': 76,
 "'": 77,
 'states': 78,
 'years': 79,
 'such': 80,
 'many': 81,
 'b

In [93]:
char2id = {"<pad>":0,"<unk>":1,"<start>":2}
for char in chars:
    char2id[char] = len(char2id)
char2id

{'<pad>': 0,
 '<unk>': 1,
 '<start>': 2,
 'e': 3,
 'a': 4,
 't': 5,
 'n': 6,
 'i': 7,
 'o': 8,
 'r': 9,
 's': 10,
 'h': 11,
 'l': 12,
 'd': 13,
 'c': 14,
 'u': 15,
 'm': 16,
 'f': 17,
 'p': 18,
 'g': 19,
 'w': 20,
 'y': 21,
 '.': 22,
 'b': 23,
 ',': 24,
 'v': 25,
 "'": 26,
 'k': 27,
 '1': 28,
 '`': 29,
 'S': 30,
 'T': 31,
 'A': 32,
 'C': 33,
 '0': 34,
 '9': 35,
 'M': 36,
 'I': 37,
 '2': 38,
 'H': 39,
 'B': 40,
 'P': 41,
 '-': 42,
 'F': 43,
 'D': 44,
 'L': 45,
 'R': 46,
 'x': 47,
 '8': 48,
 'N': 49,
 'W': 50,
 '(': 51,
 'G': 52,
 ')': 53,
 'O': 54,
 'E': 55,
 'J': 56,
 '3': 57,
 '7': 58,
 '5': 59,
 '6': 60,
 '4': 61,
 'z': 62,
 'U': 63,
 'j': 64,
 'K': 65,
 'q': 66,
 'V': 67,
 'Y': 68,
 ':': 69,
 '–': 70,
 ';': 71,
 '/': 72,
 'Z': 73,
 'é': 74,
 'Q': 75,
 'X': 76,
 '&': 77,
 '’': 78,
 '%': 79,
 '$': 80,
 '—': 81,
 '“': 82,
 ']': 83,
 '[': 84,
 '”': 85,
 'á': 86,
 'í': 87,
 '!': 88,
 '>': 89,
 '<': 90,
 'ö': 91,
 'ó': 92,
 'ü': 93,
 'š': 94,
 'ć': 95,
 '#': 96,
 'è': 97,
 '_': 98,
 'ä': 

In [94]:
# 构建特征和标签
char_datas = []
weights = []
data = datas[0]
np.array(data)

array(['Konica', 'Cup', '(', 'before', 'the', 'Minolta', 'merger', ')',
       'may', 'refer', 'to', '.'], dtype='<U7')

In [95]:
max_word_length = 16
char_data = [[char2id["<start>"]]*max_word_length] # 第一个字符是start，标志句子的开始
char_data

[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]

In [96]:
word = data[0]
word

'Konica'

In [97]:
char_word = []
for char in word:
    char_word.append(char2id.get(char,char2id["<unk>"]))
char_word

[65, 8, 6, 7, 14, 4]

In [98]:
char_word = char_word[0:max_word_length] + [char2id["<pad>"]]*(max_word_length-len(char_word))
char_word

[65, 8, 6, 7, 14, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [99]:
np.array(datas[0])

array(['Konica', 'Cup', '(', 'before', 'the', 'Minolta', 'merger', ')',
       'may', 'refer', 'to', '.'], dtype='<U7')

In [100]:
data = datas[0]
for j,word in enumerate(data):
    char_word = []
    for char in word:
        char_word.append(char2id.get(char,char2id["<unk>"]))
    char_word = char_word[0:max_word_length] + [char2id["<pad>"]]*(max_word_length-len(char_word))
    data[j] = word2id.get(data[j].lower(),word2id["<unk>"])
    char_data.append(char_word)
print (data)
print (char_data)

[56, 2, 95, 701, 100, 3, 1630, 1479, 173, 7, 143, 1]
[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [65, 8, 6, 7, 14, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [33, 15, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [23, 3, 17, 8, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 11, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [36, 7, 6, 8, 12, 5, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0], [16, 3, 9, 19, 3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [16, 4, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [9, 3, 17, 3, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [101]:
max_sentence_length = 100
weights.extend([1] * len(data)+[0]*(max_sentence_length-len(data)))
np.array(weights)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [102]:
max_sentence_length = 100
data = data[0:max_sentence_length]+[word2id["<pad>"]]*(max_sentence_length-len(data))
np.array(data)

array([  56,    2,   95,  701,  100,    3, 1630, 1479,  173,    7,  143,
          1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

In [103]:
char_data = char_data[0:max_sentence_length]+[[char2id["<pad>"]]*max_word_length]*(max_sentence_length-len(char_data))
np.array(char_data)

array([[ 2,  2,  2, ...,  2,  2,  2],
       [65,  8,  6, ...,  0,  0,  0],
       [33, 15, 18, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [104]:
import numpy as np
datas = open("./data/train.txt",encoding="utf-8").read().strip("\n").splitlines()
datas = [s.split() for s in datas]
char_datas = []
weights = []
for i,data in enumerate(datas):
    if i%1000==0:
        print (i,len(datas))
    char_data = [[char2id["<start>"]]*max_word_length]
    for j,word in enumerate(data):
        char_word = []
        for char in word:
            char_word.append(char2id.get(char,char2id["<unk>"])) # 将字符转化成id
        char_word = char_word[0:max_word_length] + \
                    [char2id["<pad>"]]*(max_word_length-len(char_word))  # 将所有词pad成相同长度
        datas[i][j] = word2id.get(datas[i][j].lower(),word2id["<unk>"]) # 将词转化为id
        char_data.append(char_word)
    weights.extend([1] * len(datas[i])+[0]*(max_sentence_length-len(datas[i])))
    datas[i] = datas[i][0:max_sentence_length]+[word2id["<pad>"]]*(max_sentence_length-len(datas[i])) # 将所有句子pad成相同长度
    char_datas.append(char_data)
    char_datas[i] = char_datas[i][0:max_sentence_length]+\
                    [[char2id["<pad>"]]*max_word_length]*(max_sentence_length-len(char_datas[i])) # 将所有句子pad成相同长度

datas = np.array(datas)
char_datas = np.array(char_datas)
weights = np.array(weights)

0 36192
1000 36192
2000 36192
3000 36192
4000 36192
5000 36192
6000 36192
7000 36192
8000 36192
9000 36192
10000 36192
11000 36192
12000 36192
13000 36192
14000 36192
15000 36192
16000 36192
17000 36192
18000 36192
19000 36192
20000 36192
21000 36192
22000 36192
23000 36192
24000 36192
25000 36192
26000 36192
27000 36192
28000 36192
29000 36192
30000 36192
31000 36192
32000 36192
33000 36192
34000 36192
35000 36192
36000 36192


In [105]:
datas

array([[   1,  505,   18, ...,    0,    0,    0],
       [   1,   13,    9, ...,    0,    0,    0],
       [  27,   10,   84, ...,    0,    0,    0],
       ...,
       [   2, 4375, 2103, ...,    0,    0,    0],
       [  46,    1,    3, ...,    0,    0,    0],
       [  56,    2,   95, ...,    0,    0,    0]])

In [106]:
char_datas

array([[[ 2,  2,  2, ...,  2,  2,  2],
        [65,  8,  6, ...,  0,  0,  0],
        [33, 15, 18, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 2,  2,  2, ...,  2,  2,  2],
        [32,  9, 14, ...,  0,  0,  0],
        [ 7, 10,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 2,  2,  2, ...,  2,  2,  2],
        [37,  5,  0, ...,  0,  0,  0],
        [20,  4, 10, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       ...,

       [[ 2,  2,  2, ...,  2,  2,  2],
        [31, 11,  3, ...,  0,  0,  0],
        [63,  6, 13, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0

In [107]:
weights

array([1, 1, 1, ..., 0, 0, 0])

In [108]:
datas = datas.reshape([-1])
char_datas = char_datas.reshape([-1,max_word_length])

In [109]:
print (datas.shape)
print (char_datas.shape)
print (weights.shape)

(3619200,)
(3619200, 16)
(3619200,)


In [110]:
# 整理成python的class用于pytorch数据读入，见pycharm